## Imports & Environment Variables

In [ ]:
from groq import Groq

GROQ_API_KEY = "INSERT GROQ API KEY HERE"

In [ ]:
#function to invoke groq response
def groq_invoke(prompt):
    
    client = Groq(
        api_key=GROQ_API_KEY,
    )
    
    chat_completion = client.chat.completions.create(
        messages = [
            {
                "role": "user",
                "content": prompt
            }
        ],
        model = "llama3-8b-8192"
    )
    
    return chat_completion.choices[0].message.content

## LLM Response No RAG

In [ ]:
#build query we want to answer
prompt = "Can you show me with code how to invoke a aws bedrock model integrated with step functions."

#get response
response = groq_invoke(prompt)
print(response)

## Internal RAG Database Building

#### Imports

In [ ]:
from langchain.schema.document import Document
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores.chroma import Chroma
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain_community.embeddings.ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

#build data paths
DATA_PATH = "./data/"
CHROMA_PATH = "./chroma/"

#### Helper Functions

In [ ]:
#function that loads in pdf documents from data path
def load_documents():
    document_loader = PyPDFDirectoryLoader(DATA_PATH)
    return document_loader.load()

In [ ]:
#helper function that splits ours documents into smaller chunks for better data processing
def split_documents(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 800,
        chunk_overlap = 80,
        length_function = len,
        is_separator_regex = False
    )
    return text_splitter.split_documents(documents)

In [ ]:
#helper function that gets our embedding functions
def get_embedding_function():
    embeddings = OllamaEmbeddings(model="nomic-embed-text")
    return embeddings

In [ ]:
#helper function to calculate chunk ids to see where the model its getting its context
def calculate_chunk_ids(chunks):
    last_page_id = None
    current_chunk_index = 0
    
    for chunk in chunks:
        source = chunk.metadata.get("source")
        page = chunk.metadata.get("page")
        current_page_id = f"{source}:{page}"
            
        if current_page_id == last_page_id:
            current_chunk_index += 1
        else:
            current_chunk_index = 0
        
        chunk_id = f"{current_page_id}:{current_chunk_index}"
        last_page_id = current_page_id
        
        chunk.metadata["id"] = chunk_id
        
    return chunks

In [ ]:
#helper function that adds embeddings to chroma database
def add_to_chroma(chunks: list[Document]):
    db = Chroma(
        persist_directory = CHROMA_PATH,
        embedding_function = get_embedding_function()
    )
    
    chunks_with_ids = calculate_chunk_ids(chunks)
    
    existing_items = db.get(include=[])
    existing_ids = set(existing_items["ids"])
    print(f"Number of existing documents in DB: {len(existing_ids)}")
    
    new_chunks = []
    for chunk in chunks_with_ids:
        if chunk.metadata["id"] not in existing_ids:
            new_chunks.append(chunk)
    
    if len(new_chunks):
        print(f"Adding new documents: {len(new_chunks)}")
        new_chunk_ids = [chunk.metadata["id"] for chunk in new_chunks]
        db.add_documents(new_chunks, ids=new_chunk_ids)
        db.persist()
    else:
        print("No new documents to add")

In [ ]:
#helper function that invokes groq with RAG
def groq_rag_invoke(query_text: str):
    #build prompt template
    PROMPT_TEMPLATE = """
    Answer the question based only on the following context:

    {context}

    ---

    Answer the question based on the above context: {question}
    """
    #get our embeddings function
    embedding_function = get_embedding_function()
    #build our chroma database
    db = Chroma(
        persist_directory=CHROMA_PATH,
        embedding_function=embedding_function
    )
    #conduct an embedding similarity search
    results = db.similarity_search_with_score(query_text, k=5)
    #get our context text
    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    #build our prompt template
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    #build our prompt with rag context
    prompt = prompt_template.format(context=context_text, question=query_text)
    #get a response from grqo with rag
    response_text = groq_invoke(prompt)
    #identify chunks used
    sources = [doc.metadata.get("id", None) for doc, _score in results]
    #format our response
    formatted_response = f"Response: {response_text}\nSource: {sources}"
    return formatted_response

## RAG Process

In [ ]:
#load in documents
documents = load_documents()
#split documents into chunks
chunks = split_documents(documents)
#add chunks to chroma db
add_to_chroma(chunks)

In [ ]:
#get rag response
rag_response = groq_rag_invoke(prompt)
print(rag_response)